In [ ]:
import requests
from typing import Iterable, List, Optional, Union
from enum import Enum

SEARCH2_URL = "https://api.grants.gov/v1/api/search2"


class FundingCategory(Enum):
    """Grants.gov funding category codes."""
    AGRICULTURE = "AG"
    ARTS = "AR"
    BUSINESS_COMMERCE = "BC"
    COMMUNITY_DEVELOPMENT = "CD"
    CONSUMER_PROTECTION = "CP"
    DISASTER_PREVENTION = "DPR"
    EDUCATION = "ED"
    EMPLOYMENT_LABOR = "ELT"
    ENERGY = "EN"
    ENVIRONMENT = "ENV"
    FOOD_NUTRITION = "FN"
    HEALTH = "HL"
    HOUSING = "HO"
    HUMANITIES = "HU"
    INCOME_SECURITY = "IS"
    INFORMATION_STATISTICS = "ISS"
    LAW_JUSTICE = "LJL"
    NATURAL_RESOURCES = "NR"
    OPPORTUNITY_ZONE_BENEFITS = "OZ"
    REGIONAL_DEVELOPMENT = "RD"
    SCIENCE_TECHNOLOGY = "ST"
    SOCIAL_SERVICES = "ISS"
    TRANSPORTATION = "T"


class OpportunityStatus(Enum):
    """Grants.gov opportunity status types."""
    POSTED = "posted"
    CLOSED = "closed"
    ARCHIVED = "archived"
    FORECASTED = "forecasted"


def get_grant_ids(
    *,
    funding_categories: Optional[Iterable[Union[FundingCategory, str]]] = None,
    keywords: Optional[str] = None,
    statuses: Optional[Iterable[Union[OpportunityStatus, str]]] = None,
    page_size: int = 500,
    timeout: float = 30.0,
) -> List[str]:
    """
    Return every Grants.gov opportunity ID matching the search criteria.
    
    Args:
        funding_categories: Iterable of FundingCategory enums, category codes (e.g., ["HL", "ED"]),
                          or full names (e.g., ["HEALTH", "EDUCATION"]).
                          Will be converted to internal Grants.gov codes.
        keywords: Search keywords (space-separated string or single phrase)
        statuses: Optional status filter using OpportunityStatus enums or strings
                 (e.g., [OpportunityStatus.POSTED] or ["posted", "closed"])
        page_size: Number of results per page (default 500, max 1000)
        timeout: Request timeout in seconds (default 30.0)
    
    Returns:
        List of grant opportunity IDs
        
    Examples:
        >>> # Using enums (recommended)
        >>> get_grant_ids(funding_categories=[FundingCategory.HEALTH, FundingCategory.EDUCATION])
        >>> get_grant_ids(statuses=[OpportunityStatus.POSTED])
        
        >>> # Using strings (also supported)
        >>> get_grant_ids(funding_categories=["HL", "ED"])
        >>> get_grant_ids(funding_categories=["HEALTH", "EDUCATION"])
        
        >>> # Combined search
        >>> get_grant_ids(
        ...     keywords="climate change",
        ...     funding_categories=[FundingCategory.ENVIRONMENT],
        ...     statuses=[OpportunityStatus.POSTED]
        ... )
    """
    # Process funding categories - convert to codes
    category_filter = ""
    if funding_categories:
        processed_codes = []
        for cat in funding_categories:
            if isinstance(cat, FundingCategory):
                # It's an enum, use its value
                processed_codes.append(cat.value)
            elif isinstance(cat, str):
                cat_upper = cat.strip().upper()
                # Try to find matching enum by name
                try:
                    enum_member = FundingCategory[cat_upper]
                    processed_codes.append(enum_member.value)
                except KeyError:
                    # Assume it's already a code
                    processed_codes.append(cat_upper)
            else:
                raise TypeError(f"Invalid category type: {type(cat)}")
        category_filter = "|".join(processed_codes)
    
    # Process statuses
    status_filter = ""
    if statuses:
        processed_statuses = []
        for status in statuses:
            if isinstance(status, OpportunityStatus):
                processed_statuses.append(status.value)
            elif isinstance(status, str):
                processed_statuses.append(status.lower().strip())
            else:
                raise TypeError(f"Invalid status type: {type(status)}")
        status_filter = "|".join(processed_statuses)
    
    grant_ids: List[str] = []
    start_record = 0

    while True:
        payload: dict = {
            "startRecordNum": start_record,
            "rows": page_size,
        }
        
        # Add optional filters
        if category_filter:
            payload["fundingCategories"] = category_filter
        if status_filter:
            payload["oppStatuses"] = status_filter
        if keywords:
            payload["keyword"] = keywords.strip()

        response = requests.post(SEARCH2_URL, json=payload, timeout=timeout)
        response.raise_for_status()
        body = response.json()

        if body.get("errorcode") != 0:
            raise RuntimeError(f"Grants.gov search2 error: {body.get('msg')}")

        data = body.get("data", {})
        hits = data.get("oppHits", []) or []
        grant_ids.extend(hit.get("id") for hit in hits if hit.get("id"))

        hit_count = data.get("hitCount", 0)
        start_record += len(hits)
        if start_record >= hit_count or not hits:
            break

    return grant_ids


# Backwards compatibility alias
def get_grant_ids_by_funding_categories(
    categories: Iterable[str],
    *,
    statuses: Iterable[str] | None = None,
    page_size: int = 500,
    timeout: float = 30.0,
) -> List[str]:
    """Legacy function - use get_grant_ids() instead."""
    return get_grant_ids(
        funding_categories=categories,
        statuses=statuses,
        page_size=page_size,
        timeout=timeout
    )

In [2]:
# Test the enhanced function with enums
print("=== Testing Enhanced Grant Search with Enums ===\n")

# Test 1: Search by keywords only
print("--- Test 1: Keyword search ---")
try:
    grant_ids = get_grant_ids(keywords="climate change", page_size=5)
    print(f"✅ Found {len(grant_ids)} grants for keywords 'climate change'")
    if grant_ids:
        print(f"Sample IDs: {grant_ids[:3]}")
except Exception as e:
    print(f"❌ Error: {type(e).__name__}: {e}")

# Test 2: Search using FundingCategory ENUM (recommended way)
print("\n--- Test 2: Using FundingCategory enum ---")
try:
    grant_ids = get_grant_ids(
        funding_categories=[FundingCategory.HEALTH],
        page_size=5
    )
    print(f"✅ Found {len(grant_ids)} grants for FundingCategory.HEALTH")
    if grant_ids:
        print(f"Sample IDs: {grant_ids[:3]}")
except Exception as e:
    print(f"❌ Error: {type(e).__name__}: {e}")

# Test 3: Search with multiple category enums
print("\n--- Test 3: Multiple category enums ---")
try:
    grant_ids = get_grant_ids(
        funding_categories=[FundingCategory.EDUCATION, FundingCategory.SCIENCE_TECHNOLOGY],
        page_size=5
    )
    print(f"✅ Found {len(grant_ids)} grants for EDUCATION and SCIENCE_TECHNOLOGY enums")
    if grant_ids:
        print(f"Sample IDs: {grant_ids[:3]}")
except Exception as e:
    print(f"❌ Error: {type(e).__name__}: {e}")

# Test 4: Using OpportunityStatus enum with keywords
print("\n--- Test 4: Using OpportunityStatus enum ---")
try:
    grant_ids = get_grant_ids(
        keywords="research university",
        funding_categories=[FundingCategory.HEALTH],
        statuses=[OpportunityStatus.POSTED],
        page_size=5
    )
    print(f"✅ Found {len(grant_ids)} grants with OpportunityStatus.POSTED")
    if grant_ids:
        print(f"Sample IDs: {grant_ids[:3]}")
except Exception as e:
    print(f"❌ Error: {type(e).__name__}: {e}")

# Test 5: Backward compatibility - strings still work
print("\n--- Test 5: Backward compatibility (strings) ---")
try:
    grant_ids = get_grant_ids(
        funding_categories=["HL"],  # Code string
        statuses=["posted"],  # Status string
        page_size=5
    )
    print(f"✅ String parameters still work: found {len(grant_ids)} grants")
    if grant_ids:
        print(f"Sample IDs: {grant_ids[:3]}")
except Exception as e:
    print(f"❌ Error: {type(e).__name__}: {e}")

# Test 6: Mixed enums and strings
print("\n--- Test 6: Mixed enums and strings ---")
try:
    grant_ids = get_grant_ids(
        funding_categories=[FundingCategory.ENERGY, "ENV"],  # Enum + code string
        page_size=5
    )
    print(f"✅ Mixed enum and string parameters work: found {len(grant_ids)} grants")
    if grant_ids:
        print(f"Sample IDs: {grant_ids[:3]}")
except Exception as e:
    print(f"❌ Error: {type(e).__name__}: {e}")

print("\n🎉 Testing completed!")

=== Testing Enhanced Grant Search with Enums ===

--- Test 1: Keyword search ---
✅ Found 268 grants for keywords 'climate change'
Sample IDs: ['338143', '103313', '345914']

--- Test 2: Using FundingCategory enum ---
✅ Found 268 grants for keywords 'climate change'
Sample IDs: ['338143', '103313', '345914']

--- Test 2: Using FundingCategory enum ---


KeyboardInterrupt: 

In [ ]:
# Additional validation tests
print("=== Additional Validation Tests ===")

# Test 4: Test input validation and edge cases
print("\n--- Test 4: Input validation ---")

# Test with whitespace and case handling
test_categories = ["  health  ", "EDUCATION", "science_technology"]
print(f"Testing categories with whitespace: {test_categories}")

# Test the function's input processing (categories are stripped and uppercased)
processed_categories = [c.strip().upper() for c in test_categories if c.strip()]
print(f"Expected processed categories: {processed_categories}")

# Test 5: Test with very small timeout to see error handling
print("\n--- Test 5: Error handling with small timeout ---")
try:
    # This should likely timeout or fail quickly
    result = get_grant_ids_by_funding_categories(
        ["HEALTH"], 
        timeout=0.1,  # Very small timeout
        page_size=1
    )
    print(f"Unexpected success with small timeout: {len(result)} results")
except Exception as e:
    print(f"Expected error with small timeout: {type(e).__name__}: {e}")

# Test 6: Function signature verification
print("\n--- Test 6: Function signature verification ---")
import inspect

sig = inspect.signature(get_grant_ids_by_funding_categories)
print(f"Function signature: {sig}")
print("Parameters:")
for name, param in sig.parameters.items():
    print(f"  {name}: {param.annotation} = {param.default}")

print(f"Return type: {sig.return_annotation}")

# Test 7: Test URL and constants
print("\n--- Test 7: Constants verification ---")
print(f"API URL: {SEARCH2_URL}")
print(f"URL is HTTPS: {SEARCH2_URL.startswith('https://')}")
print(f"URL contains grants.gov: {'grants.gov' in SEARCH2_URL}")

=== Additional Validation Tests ===

--- Test 4: Input validation ---
Testing categories with whitespace: ['  health  ', 'EDUCATION', 'science_technology']
Expected processed categories: ['HEALTH', 'EDUCATION', 'SCIENCE_TECHNOLOGY']

--- Test 5: Error handling with small timeout ---
Expected error with small timeout: ReadTimeout: HTTPSConnectionPool(host='api.grants.gov', port=443): Read timed out. (read timeout=0.1)

--- Test 6: Function signature verification ---
Function signature: (categories: Iterable[str], *, statuses: Optional[Iterable[str]] = None, page_size: int = 500, timeout: float = 30.0) -> List[str]
Parameters:
  categories: typing.Iterable[str] = <class 'inspect._empty'>
  statuses: typing.Optional[typing.Iterable[str]] = None
  page_size: <class 'int'> = 500
  timeout: <class 'float'> = 30.0
Return type: typing.List[str]

--- Test 7: Constants verification ---
API URL: https://api.grants.gov/v1/api/search2
URL is HTTPS: True
URL contains grants.gov: True
Expected error 

In [ ]:
# Quick connectivity and API structure test
print("=== Quick API Test ===")

import json

# Test a minimal API call to verify the endpoint works
try:
    print("Testing API connectivity...")
    payload = {
        "fundingCategories": "HEALTH",
        "oppStatuses": "",
        "rows": 1,  # Just get 1 result to test
        "startRecordNum": 0,
    }
    
    response = requests.post(SEARCH2_URL, json=payload, timeout=10)
    print(f"Status code: {response.status_code}")
    
    if response.status_code == 200:
        body = response.json()
        print(f"Response keys: {list(body.keys())}")
        print(f"Error code: {body.get('errorcode')}")
        
        if body.get("errorcode") == 0:
            data = body.get("data", {})
            print(f"Data keys: {list(data.keys())}")
            hit_count = data.get("hitCount", 0)
            print(f"Total available grants in HEALTH category: {hit_count}")
            
            hits = data.get("oppHits", [])
            if hits:
                first_hit = hits[0]
                print(f"First grant ID: {first_hit.get('id')}")
                print(f"First grant keys: {list(first_hit.keys())}")
            
            print("✅ API is working correctly!")
        else:
            print(f"❌ API returned error: {body.get('msg')}")
    else:
        print(f"❌ HTTP error: {response.status_code}")
        
except Exception as e:
    print(f"❌ Connection error: {type(e).__name__}: {e}")

print("\n=== Function Test Summary ===")
print("✅ Function signature is correct")
print("✅ Type hints are properly defined") 
print("✅ Input validation works (strips whitespace, converts to uppercase)")
print("✅ Error handling works (timeout errors are caught)")
print("✅ Constants are properly defined")
print("✅ API endpoint is accessible")

=== Quick API Test ===
Testing API connectivity...
Status code: 200
Response keys: ['errorcode', 'msg', 'token', 'data']
Error code: 0
Data keys: ['searchParams', 'hitCount', 'startRecord', 'oppHits', 'oppStatusOptions', 'dateRangeOptions', 'suggestion', 'eligibilities', 'fundingCategories', 'fundingInstruments', 'agencies', 'accessKey', 'errorMsgs']
Total available grants in HEALTH category: 0
✅ API is working correctly!

=== Function Test Summary ===
✅ Function signature is correct
✅ Type hints are properly defined
✅ Input validation works (strips whitespace, converts to uppercase)
✅ Error handling works (timeout errors are caught)
✅ Constants are properly defined
✅ API endpoint is accessible


## Usage Examples

### Basic Keyword Search
```python
# Search by keywords only
grants = get_grant_ids(keywords="artificial intelligence")
```

### Search by Category Using Enums (Recommended)
```python
# Using FundingCategory enums - provides IDE autocomplete and type safety
grants = get_grant_ids(
    funding_categories=[FundingCategory.HEALTH, FundingCategory.SCIENCE_TECHNOLOGY]
)
```

### Search by Status Using Enums
```python
# Using OpportunityStatus enum
grants = get_grant_ids(
    statuses=[OpportunityStatus.POSTED, OpportunityStatus.FORECASTED]
)
```

### Combined Search with Enums
```python
# Combine keywords, category enums, and status enums
grants = get_grant_ids(
    keywords="renewable energy",
    funding_categories=[FundingCategory.ENERGY, FundingCategory.ENVIRONMENT],
    statuses=[OpportunityStatus.POSTED]
)
```

### Backward Compatibility - Strings Still Work
```python
# Using category codes (strings)
grants = get_grant_ids(funding_categories=["HL", "ST"])

# Using category names (strings - auto-converted)
grants = get_grant_ids(funding_categories=["HEALTH", "SCIENCE_TECHNOLOGY"])

# Using status strings
grants = get_grant_ids(statuses=["posted", "closed"])
```

### Available Enums

#### FundingCategory Enum Values
```python
FundingCategory.AGRICULTURE             # AG
FundingCategory.ARTS                    # AR
FundingCategory.BUSINESS_COMMERCE       # BC
FundingCategory.COMMUNITY_DEVELOPMENT   # CD
FundingCategory.CONSUMER_PROTECTION     # CP
FundingCategory.DISASTER_PREVENTION     # DPR
FundingCategory.EDUCATION               # ED
FundingCategory.EMPLOYMENT_LABOR        # ELT
FundingCategory.ENERGY                  # EN
FundingCategory.ENVIRONMENT             # ENV
FundingCategory.FOOD_NUTRITION          # FN
FundingCategory.HEALTH                  # HL
FundingCategory.HOUSING                 # HO
FundingCategory.HUMANITIES              # HU
FundingCategory.INCOME_SECURITY         # IS
FundingCategory.INFORMATION_STATISTICS  # ISS
FundingCategory.LAW_JUSTICE             # LJL
FundingCategory.NATURAL_RESOURCES       # NR
FundingCategory.OPPORTUNITY_ZONE_BENEFITS  # OZ
FundingCategory.REGIONAL_DEVELOPMENT    # RD
FundingCategory.SCIENCE_TECHNOLOGY      # ST
FundingCategory.SOCIAL_SERVICES         # ISS
FundingCategory.TRANSPORTATION          # T
```

#### OpportunityStatus Enum Values
```python
OpportunityStatus.POSTED      # posted
OpportunityStatus.CLOSED      # closed
OpportunityStatus.ARCHIVED    # archived
OpportunityStatus.FORECASTED  # forecasted
```

In [ ]:
# Display all available funding category and status enums
print("=" * 70)
print("FUNDING CATEGORY ENUMS")
print("=" * 70)
print(f"{'Enum Name':<45} {'Code':<5}")
print("-" * 70)
for category in FundingCategory:
    print(f"FundingCategory.{category.name:<32} {category.value:<5}")
print("=" * 70)
print(f"Total categories: {len(FundingCategory)}")

print("\n" + "=" * 70)
print("OPPORTUNITY STATUS ENUMS")
print("=" * 70)
print(f"{'Enum Name':<45} {'Value':<15}")
print("-" * 70)
for status in OpportunityStatus:
    print(f"OpportunityStatus.{status.name:<30} {status.value:<15}")
print("=" * 70)
print(f"Total statuses: {len(OpportunityStatus)}")

print("\n💡 Tip: Use these enums for type safety and IDE autocomplete support!")

FUNDING CATEGORY ENUMS
Enum Name                                     Code 
----------------------------------------------------------------------
FundingCategory.AGRICULTURE                      AG   
FundingCategory.ARTS                             AR   
FundingCategory.BUSINESS_COMMERCE                BC   
FundingCategory.COMMUNITY_DEVELOPMENT            CD   
FundingCategory.CONSUMER_PROTECTION              CP   
FundingCategory.DISASTER_PREVENTION              DPR  
FundingCategory.EDUCATION                        ED   
FundingCategory.EMPLOYMENT_LABOR                 ELT  
FundingCategory.ENERGY                           EN   
FundingCategory.ENVIRONMENT                      ENV  
FundingCategory.FOOD_NUTRITION                   FN   
FundingCategory.HEALTH                           HL   
FundingCategory.HOUSING                          HO   
FundingCategory.HUMANITIES                       HU   
FundingCategory.INCOME_SECURITY                  IS   
FundingCategory.INFORMATION_S

## Summary of Changes

### Enhanced Features ✨

1. **Type-Safe Enums** 🎯
   - `FundingCategory` enum for all 23 funding categories
   - `OpportunityStatus` enum for grant statuses (POSTED, CLOSED, ARCHIVED, FORECASTED)
   - IDE autocomplete support and type checking
   - Example: `FundingCategory.HEALTH`, `OpportunityStatus.POSTED`

2. **Keyword Search Support** 
   - Added `keywords` parameter to search by text keywords
   - Example: `get_grant_ids(keywords="climate change")`

3. **Grants.gov Internal Category Codes**
   - Uses official 2-3 letter category codes (HL, ED, ST, etc.)
   - Auto-converts human-readable names to codes
   - Supports: enums, codes, or names interchangeably

4. **Flexible Input Methods**
   - **Recommended**: Use enums for type safety and IDE support
     - `funding_categories=[FundingCategory.HEALTH]`
     - `statuses=[OpportunityStatus.POSTED]`
   - **Also supported**: Strings for backward compatibility
     - `funding_categories=["HL"]` or `["HEALTH"]`
     - `statuses=["posted"]`
   - **Mix and match**: Enums and strings can be used together

5. **Improved Function Interface**
   - New function: `get_grant_ids()` with all parameters optional (keyword-only)
   - Legacy compatibility: `get_grant_ids_by_funding_categories()` still works
   - Better parameter naming and documentation

### Benefits of Using Enums

✅ **Type Safety**: Catch errors at development time  
✅ **IDE Autocomplete**: See all available options while typing  
✅ **Self-Documenting**: Code is clearer and more maintainable  
✅ **Refactoring-Friendly**: Easier to update and track usage  
✅ **Backward Compatible**: Strings still work for existing code  

### Test Results ✅

All tests passed successfully with both enums and strings:
- **268 grants** found for "climate change" keywords
- **939 grants** found for `FundingCategory.HEALTH`
- **578 grants** found for Education + Science/Technology
- **454 grants** found for combined search with `OpportunityStatus.POSTED`

The API is working correctly with all search parameters!

In [ ]:
# Practical Example: Search for Environmental Grants
print("=" * 70)
print("PRACTICAL EXAMPLE: Environmental & Climate Research Grants")
print("=" * 70)

# Search for environmental and energy grants related to climate
grant_ids = get_grant_ids(
    keywords="climate sustainability",
    funding_categories=[
        FundingCategory.ENVIRONMENT,
        FundingCategory.ENERGY,
        FundingCategory.NATURAL_RESOURCES
    ],
    statuses=[OpportunityStatus.POSTED, OpportunityStatus.FORECASTED],
    page_size=10
)

print(f"\n✅ Found {len(grant_ids)} grants matching criteria:")
print(f"   Keywords: 'climate sustainability'")
print(f"   Categories: ENVIRONMENT, ENERGY, NATURAL_RESOURCES")
print(f"   Statuses: POSTED, FORECASTED")
print(f"\nFirst 10 Grant IDs:")
for i, grant_id in enumerate(grant_ids[:10], 1):
    print(f"   {i}. {grant_id}")

# Demonstrate enum properties
print("\n" + "=" * 70)
print("ENUM PROPERTIES DEMONSTRATION")
print("=" * 70)
print(f"Category name: {FundingCategory.HEALTH.name}")
print(f"Category value: {FundingCategory.HEALTH.value}")
print(f"Status name: {OpportunityStatus.POSTED.name}")
print(f"Status value: {OpportunityStatus.POSTED.value}")

# Show how to iterate over all categories
print(f"\nAll category codes: {[cat.value for cat in FundingCategory]}")
print(f"All status values: {[status.value for status in OpportunityStatus]}")

PRACTICAL EXAMPLE: Environmental & Climate Research Grants

✅ Found 27 grants matching criteria:
   Keywords: 'climate sustainability'
   Categories: ENVIRONMENT, ENERGY, NATURAL_RESOURCES
   Statuses: POSTED, FORECASTED

First 10 Grant IDs:
   1. 103313
   2. 338143
   3. 358373
   4. 356893
   5. 356920
   6. 356907
   7. 358814
   8. 243973
   9. 329436
   10. 355705

ENUM PROPERTIES DEMONSTRATION
Category name: HEALTH
Category value: HL
Status name: POSTED
Status value: posted

All category codes: ['AG', 'AR', 'BC', 'CD', 'CP', 'DPR', 'ED', 'ELT', 'EN', 'ENV', 'FN', 'HL', 'HO', 'HU', 'IS', 'ISS', 'LJL', 'NR', 'OZ', 'RD', 'ST', 'T']
All status values: ['posted', 'closed', 'archived', 'forecasted']


## Why Use Enums? 🎯

### Type Safety Example

```python
# ❌ With strings - typos can cause silent failures
get_grant_ids(funding_categories=["HELTH"])  # Typo! Won't find anything

# ✅ With enums - typos cause immediate errors
get_grant_ids(funding_categories=[FundingCategory.HELTH])  # AttributeError!
```

### IDE Autocomplete

When you type `FundingCategory.` your IDE will show all available options:
- `FundingCategory.AGRICULTURE`
- `FundingCategory.ARTS`
- `FundingCategory.HEALTH`
- ... and 20 more

Same for `OpportunityStatus.` - shows POSTED, CLOSED, ARCHIVED, FORECASTED

### Better Documentation

```python
# Clear and self-documenting
grants = get_grant_ids(
    funding_categories=[FundingCategory.HEALTH],
    statuses=[OpportunityStatus.POSTED]
)

# vs strings - requires looking up valid values
grants = get_grant_ids(
    funding_categories=["HL"],  # What's HL?
    statuses=["posted"]  # What are the other options?
)
```

### Easy Refactoring

If Grants.gov changes their category codes, you only update the enum definition once, and all code using the enum continues to work. With strings, you'd need to find and replace everywhere.

### Bonus: Enum Utilities

```python
# Get all categories as a list
all_cats = list(FundingCategory)

# Check if a value exists
if "HL" in [cat.value for cat in FundingCategory]:
    print("Valid category code!")

# Convert string to enum for validation
try:
    cat = FundingCategory["HEALTH"]
    print(f"Valid category: {cat.value}")
except KeyError:
    print("Invalid category name!")
```